# Topic 2A: Climate and the Environment, General Measurements and Statistics

# Project Purpose: We want to understand if we can predict air quality at a particular location based off of weather and greenhouse gas emissions in a specified radius of that location

## EDA



## Important preliminary information:
### AQI is measured per specific pollutants, PM2.5, PM10, Ozone, CO, SO2, NO2. With the overall Daily AQI being the maximum value of any specific pollutant AQI.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [3]:
#aqi_by_county = pd.read_csv("county_aqi_2020_daily.csv")
daily_global_weather_2020 = pd.read_csv("datasets/2020_daily_global_weather.csv").drop(columns="Unnamed: 0")
us_greenhouse_gas_emission_direct_emitter_gas_type = pd.read_csv("datasets/greenhouse_gas_emmiter_gas_types.csv", low_memory=False)

In [4]:
aqi_sites = pd.read_csv("datasets/aqs_sites.csv")

daily_county_aqi = pd.read_csv("datasets/county_aqi_2020_daily.csv") 

daily_temp = pd.read_csv("datasets/daily_temperature_2020.csv")

daily_wind = pd.read_csv("datasets/daily_wind_2020.csv")

traffic_volumes = pd.read_csv("datasets/traffic_volumes.csv")

## "AQI by county" shows the annual distribution of AQI and ratings by State and County. Each row represents the AQI distribution of a State and County pair.

## "daily global weather 2020" shows the daily average temperature and precipitation on a global scale. Each row represents a specific station somewhere in the world per day.

## "US greenhouse gas emission" shows CO2 equivalent green house gas emissions for various greehhouse gasses by state, county, city and year. Each row represents a CO2 equivalent green house gas emission for a speific gas in a city by year.

## "aqi sites" has meta deta about the AQI sites where AQI values are measured and determined. Each row is a site.

## "daily_county_aqi" shows the daily AQI and its defining parameter for a State and County. Each row represents a day for each state and county pair.

## "daily_temp" shows the average daily temperature for a state and county. Each row represents a day for each state and county pair. (City column exists but does not apply to all rows)

## "daily_wind" shows the average wind speed and wind direction for a state and county. Each row represents a day for each state and county pair. (City column exists but does not apply to all rows)

In [5]:
# We want to predict the AQI column
daily_county_aqi

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2020-01-01,48,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2020-01-04,13,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2020-01-07,14,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2020-01-10,39,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2020-01-13,29,Good,PM2.5,01-003-0010,1
...,...,...,...,...,...,...,...,...,...,...
336696,Wyoming,Weston,56,45,2020-12-27,32,Good,Ozone,56-045-0003,2
336697,Wyoming,Weston,56,45,2020-12-28,30,Good,Ozone,56-045-0003,2
336698,Wyoming,Weston,56,45,2020-12-29,33,Good,Ozone,56-045-0003,2
336699,Wyoming,Weston,56,45,2020-12-30,33,Good,Ozone,56-045-0003,2


## Looking at the data sets, we have different granularities. For example, us_greenhouse_gas_emissions shows the ANNUAL CO2 equivalent emissions for a particular GH gas by State, County and City over multiple years. While the daily_no2, daily_so2, daily_co, daily_ozone, daily_temp, daily_wind, and daily_global_weather_2020 feature DAILY measurements over 1 year.


## Since we are trying to predict AQI, we should not include the values used to calculate the AQI found within daily_co, daily_ozone, daily_no2, daily_so2.

## Tentative Features to include in the model: daily_temp, daily_wind_speed, daily_wind_direction, daily_precipitation,  elevation, gh_gas_emissions by gas (co2 equivalents). Each row will be the daily value by each state and county.

In [6]:
#PRCP is average percipitation in tenths of a mm, TAVG is tempearture average at each station for each day
daily_global_weather_2020

,Station,Date,TAVG,Latitude,Longitude,Elevation,PRCP
0,AE000041196,2020-01-01,211.0,25.3330,55.5170,34.0,0.0
1,AEM00041194,2020-01-01,217.0,25.2550,55.3640,10.4,0.0
2,AFM00040938,2020-01-01,54.0,34.2100,62.2280,977.2,23.0
3,AG000060611,2020-01-01,71.0,28.0500,9.6331,561.0,10.0
4,AGE00147708,2020-01-01,99.0,36.7200,4.0500,222.0,0.0
...,...,...,...,...,...,...,...
1064278,USW00094847,2020-10-22,96.0,42.2314,-83.3308,192.3,157.0
1064279,USW00094849,2020-10-22,67.0,45.0717,-83.5644,208.5,99.0
1064280,USW00094860,2020-10-22,77.0,42.8939,-85.5447,237.1,165.0
1064281,USW00094908,2020-10-22,73.0,42.3978,-90.7036,321.9,361.0


In [7]:
us_greenhouse_gas_emission_direct_emitter_gas_type.columns

Index(['V_GHG_EMITTER_GAS.ADDRESS1', 'V_GHG_EMITTER_GAS.ADDRESS2',
       'V_GHG_EMITTER_GAS.CITY', 'V_GHG_EMITTER_GAS.CO2E_EMISSION',
       'V_GHG_EMITTER_GAS.COUNTY', 'V_GHG_EMITTER_GAS.FACILITY_ID',
       'V_GHG_EMITTER_GAS.GAS_CODE', 'V_GHG_EMITTER_GAS.GAS_NAME',
       'V_GHG_EMITTER_GAS.LATITUDE', 'V_GHG_EMITTER_GAS.LONGITUDE',
       'V_GHG_EMITTER_GAS.STATE', 'V_GHG_EMITTER_GAS.STATE_NAME',
       'V_GHG_EMITTER_GAS.YEAR', 'V_GHG_EMITTER_GAS.ZIP',
       'V_GHG_EMITTER_GAS.FACILITY_NAME', 'V_GHG_EMITTER_GAS.COUNTY_FIPS'],
      dtype='object')

In [9]:
daily_county_aqi.columns

Index(['State Name', 'county Name', 'State Code', 'County Code', 'Date', 'AQI',
       'Category', 'Defining Parameter', 'Defining Site',
       'Number of Sites Reporting'],
      dtype='object')

In [10]:
aqi_sites.columns

Index(['State Code', 'County Code', 'Site Number', 'Latitude', 'Longitude',
       'Datum', 'Elevation', 'Land Use', 'Location Setting',
       'Site Established Date', 'Site Closed Date', 'Met Site State Code',
       'Met Site County Code', 'Met Site Site Number', 'Met Site Type',
       'Met Site Distance', 'Met Site Direction', 'GMT Offset',
       'Owning Agency', 'Local Site Name', 'Address', 'Zip Code', 'State Name',
       'County Name', 'City Name', 'CBSA Name', 'Tribe Name',
       'Extraction Date'],
      dtype='object')

## The defining site column in daily_county_aqi is defined by "State Code"-"County Code"-"Site Number" for a given State and County in aqi_sites. Thus, the City name can be determined by the defining site using the aqi_sites table, if increased granularity is required.

In [11]:
daily_wind.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'Arithmetic Mean',
       '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name',
       'CBSA Name', 'Date of Last Change'],
      dtype='object')

In [12]:
daily_temp.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'Arithmetic Mean',
       '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name',
       'CBSA Name', 'Date of Last Change'],
      dtype='object')

# Aggregating Daily temp, wind speed and wind direction by state and county

## EDA on Daily Temperature Data

In [13]:
# EDA on daily temperature, wind speed and wind direction

# Extracting relevant columns
truncated_daily_temp = daily_temp[["State Code", "County Code", "Site Num", "State Name", "County Name", "City Name", "Latitude", "Longitude", "Date Local", "Arithmetic Mean"]]

# Array of US States and territories
US_States_territories = us_greenhouse_gas_emission_direct_emitter_gas_type["V_GHG_EMITTER_GAS.STATE_NAME"].unique()

# Only US States and territories should be in our data
truncated_daily_temp = truncated_daily_temp[truncated_daily_temp["State Name"].str.upper().isin(US_States_territories)]

# Impute missing 0s, with mean of the arithmetic mean, there are no null values
truncated_daily_temp["Arithmetic Mean"] = truncated_daily_temp["Arithmetic Mean"].replace(0, truncated_daily_temp["Arithmetic Mean"].mean())

# Rename Arithmetic Mean column to include temperature and units (Temp - F)
truncated_daily_temp = truncated_daily_temp.rename(columns={"Arithmetic Mean": "Mean Temperature (F)"})

In [14]:
truncated_daily_temp

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Temperature (F)
0,1,53,1000,Alabama,Escambia,Not in a city,31.09210,-87.54350,2020-01-01,53.540000
1,1,53,1000,Alabama,Escambia,Not in a city,31.09210,-87.54350,2020-01-02,67.342857
2,1,53,1000,Alabama,Escambia,Not in a city,31.09210,-87.54350,2020-01-03,26.580000
3,1,53,1000,Alabama,Escambia,Not in a city,31.09210,-87.54350,2020-01-04,44.830000
4,1,53,1000,Alabama,Escambia,Not in a city,31.09210,-87.54350,2020-01-05,51.550000
...,...,...,...,...,...,...,...,...,...,...
290326,56,45,800,Wyoming,Weston,Newcastle,43.84539,-104.20512,2020-12-27,31.000000
290327,56,45,800,Wyoming,Weston,Newcastle,43.84539,-104.20512,2020-12-28,26.000000
290328,56,45,800,Wyoming,Weston,Newcastle,43.84539,-104.20512,2020-12-29,22.958333
290329,56,45,800,Wyoming,Weston,Newcastle,43.84539,-104.20512,2020-12-30,14.583333


## EDA on Daily Wind Data

In [15]:
# Extract relevant Columns
truncated_daily_wind = daily_wind[["State Code", "County Code", "Site Num", "State Name", "County Name", "City Name", "Latitude", "Longitude", "Parameter Name", "Date Local", "Arithmetic Mean"]]

# Only US States and territories should be in our data
truncated_daily_wind = truncated_daily_wind[truncated_daily_wind["State Name"].str.upper().isin(US_States_territories)]

# Seperate wind direction and wind speed before imputing missing values
daily_wind_speed = truncated_daily_wind[truncated_daily_wind["Parameter Name"] == "Wind Speed - Resultant"].reset_index(drop=True)

daily_wind_direction = truncated_daily_wind[truncated_daily_wind["Parameter Name"] == "Wind Direction - Resultant"].reset_index(drop=True)

# Impute missing values or 0 values with mean of the column
daily_wind_speed["Arithmetic Mean"] = daily_wind_speed["Arithmetic Mean"].replace(0, daily_wind_speed["Arithmetic Mean"].mean())

daily_wind_direction["Arithmetic Mean"] = daily_wind_direction["Arithmetic Mean"].replace(0, daily_wind_direction["Arithmetic Mean"].mean())

# Rename Arithmetic Mean columns to include the measurement and units
daily_wind_speed = daily_wind_speed.rename(columns={"Arithmetic Mean": "Mean Wind Speed (knots)"}).drop(columns=["Parameter Name"])
daily_wind_direction = daily_wind_direction.rename(columns={"Arithmetic Mean": "Mean Wind Direction (compass degrees)"}).drop(columns=["Parameter Name"])

In [16]:
daily_wind_speed

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Wind Speed (knots)
0,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-01,4.349492
1,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-02,4.349492
2,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-03,4.349492
3,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-04,4.349492
4,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-05,4.349492
...,...,...,...,...,...,...,...,...,...,...
239426,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-27,1.795833
239427,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-28,1.212500
239428,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-29,1.441667
239429,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-30,1.466667


In [17]:
daily_wind_direction

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Wind Direction (compass degrees)
0,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-01,179.900000
1,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-02,108.857143
2,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-03,271.000000
3,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-04,230.100000
4,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-05,251.100000
...,...,...,...,...,...,...,...,...,...,...
242333,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-27,190.375000
242334,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-28,171.375000
242335,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-29,181.166667
242336,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-30,173.666667


## Merging the wind, and temperature data

In [18]:
wind_data_merge = pd.merge(daily_wind_direction, daily_wind_speed, how='inner', on=["State Code", "County Code", "Site Num", "State Name", "County Name", "City Name", "Latitude", "Longitude", "Date Local"])

In [19]:
weather_data = truncated_daily_temp.merge(wind_data_merge, how='inner', on=["State Code", "County Code", "Site Num", "State Name", "County Name", "City Name", "Latitude", "Longitude", "Date Local"])

## Final dataset which includes weather data

In [100]:
weather_data

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Temperature (F),Mean Wind Direction (compass degrees),Mean Wind Speed (knots)
0,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-01,53.540000,179.900000,4.349492
1,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-02,67.342857,108.857143,4.349492
2,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-03,26.580000,271.000000,4.349492
3,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-04,44.830000,230.100000,4.349492
4,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-05,51.550000,251.100000,4.349492
...,...,...,...,...,...,...,...,...,...,...,...,...
228161,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-27,15.250000,190.375000,1.795833
228162,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-28,-4.833333,171.375000,1.212500
228163,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-29,-1.625000,181.166667,1.441667
228164,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-30,8.291667,173.666667,1.466667


In [93]:
len(weather_data["City Name"].unique())

368

# Incorporating land use, elevation and location setting for the AQI sites into weather data set as features

In [76]:
truncated_aqi_sites = aqi_sites[["State Code", "County Code", "Site Number", "State Name", "County Name", "City Name", "Elevation", "Land Use", "Location Setting"]].rename(columns={"Site Number": "Site Num"})

# Only showing US and territories aqi sites
truncated_aqi_sites = truncated_aqi_sites[truncated_aqi_sites["State Name"].str.upper().isin(US_States_territories)]

truncated_aqi_sites["State Code"] = truncated_aqi_sites["State Code"].astype(int)

In [84]:
truncated_aqi_sites[(truncated_aqi_sites["State Name"] == "Alabama") & (truncated_aqi_sites["County Name"] == "Escambia")]

,State Code,County Code,Site Num,State Name,County Name,City Name,Elevation,Land Use,Location Setting
80,1,53,1,Alabama,Escambia,Brewton,37.00,RESIDENTIAL,SUBURBAN
81,1,53,2,Alabama,Escambia,Brewton,50.00,COMMERCIAL,URBAN AND CENTER CITY
82,1,53,1000,Alabama,Escambia,Not in a City,89.61,AGRICULTURAL,RURAL


In [111]:
feature_set = weather_data.merge(truncated_aqi_sites, on=["State Code", "County Code", "Site Num"])

In [112]:
feature_set = feature_set.drop(columns=["State Name_y", "County Name_y", "City Name_y"]).rename(columns={"State Name_x": "State Name", "County Name_x": "County Name", "City Name_x": "City Name"})

In [113]:
feature_set

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Temperature (F),Mean Wind Direction (compass degrees),Mean Wind Speed (knots),Elevation,Land Use,Location Setting
0,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-01,53.540000,179.900000,4.349492,89.61,AGRICULTURAL,RURAL
1,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-02,67.342857,108.857143,4.349492,89.61,AGRICULTURAL,RURAL
2,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-03,26.580000,271.000000,4.349492,89.61,AGRICULTURAL,RURAL
3,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-04,44.830000,230.100000,4.349492,89.61,AGRICULTURAL,RURAL
4,1,53,1000,Alabama,Escambia,Not in a city,31.092100,-87.543500,2020-01-05,51.550000,251.100000,4.349492,89.61,AGRICULTURAL,RURAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228161,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-27,15.250000,190.375000,1.795833,2230.00,FOREST,RURAL
228162,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-28,-4.833333,171.375000,1.212500,2230.00,FOREST,RURAL
228163,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-29,-1.625000,181.166667,1.441667,2230.00,FOREST,RURAL
228164,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-12-30,8.291667,173.666667,1.466667,2230.00,FOREST,RURAL


## Adding the observed values for AQI to the features dataset

In [122]:
## Extracting the Site Num from Defining Site
daily_county_aqi["Site Num"] = daily_county_aqi["Defining Site"].str.extract(pat=f'\d+-\d+-(\d+)').astype(int)

In [129]:
## Renaming columns to allow for merge
daily_county_aqi = daily_county_aqi.rename(columns={"county Name": "County Name", "Date": "Date Local"})

In [133]:
## Merging with the feature set
final_data_set = feature_set.merge(daily_county_aqi, on=["State Code", "County Code", "Site Num", "Date Local"])

In [137]:
## Dropping redundant columns and renaming 
final_data_set = final_data_set.drop(columns=["State Name_y", "County Name_y"]).rename(columns={"State Name_x": "State Name", "County Name_x":"County Name"})

In [139]:
final_data_set

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Temperature (F),Mean Wind Direction (compass degrees),Mean Wind Speed (knots),Elevation,Land Use,Location Setting,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.815000,2020-01-09,52.125000,137.125000,4.904167,177.0,COMMERCIAL,URBAN AND CENTER CITY,56,Moderate,PM2.5,01-073-0023,9
1,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.815000,2020-01-10,65.750000,163.708333,7.583333,177.0,COMMERCIAL,URBAN AND CENTER CITY,44,Good,PM2.5,01-073-0023,9
2,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.815000,2020-01-14,63.937500,208.916667,3.166667,177.0,COMMERCIAL,URBAN AND CENTER CITY,30,Good,NO2,01-073-0023,9
3,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.815000,2020-01-16,58.012500,75.458333,5.945833,177.0,COMMERCIAL,URBAN AND CENTER CITY,31,Good,Ozone,01-073-0023,9
4,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.815000,2020-01-17,54.945833,105.375000,5.491667,177.0,COMMERCIAL,URBAN AND CENTER CITY,34,Good,Ozone,01-073-0023,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94814,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-10-05,45.708333,162.666667,2.287500,2230.0,FOREST,RURAL,54,Moderate,PM2.5,56-039-1013,3
94815,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-10-07,44.083333,174.791667,2.087500,2230.0,FOREST,RURAL,85,Moderate,PM2.5,56-039-1013,3
94816,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-10-08,46.375000,169.375000,3.016667,2230.0,FOREST,RURAL,69,Moderate,PM2.5,56-039-1013,3
94817,56,39,1013,Wyoming,Teton,Not in a city,44.373056,-110.830833,2020-10-09,45.041667,168.666667,2.612500,2230.0,FOREST,RURAL,55,Moderate,PM2.5,56-039-1013,4


# US Emissions data set
## Incorporating US GH emissions by state/county as features 

In [142]:
us_greenhouse_gas_emission_direct_emitter_gas_type.columns

Index(['V_GHG_EMITTER_GAS.ADDRESS1', 'V_GHG_EMITTER_GAS.ADDRESS2',
       'V_GHG_EMITTER_GAS.CITY', 'V_GHG_EMITTER_GAS.CO2E_EMISSION',
       'V_GHG_EMITTER_GAS.COUNTY', 'V_GHG_EMITTER_GAS.FACILITY_ID',
       'V_GHG_EMITTER_GAS.GAS_CODE', 'V_GHG_EMITTER_GAS.GAS_NAME',
       'V_GHG_EMITTER_GAS.LATITUDE', 'V_GHG_EMITTER_GAS.LONGITUDE',
       'V_GHG_EMITTER_GAS.STATE', 'V_GHG_EMITTER_GAS.STATE_NAME',
       'V_GHG_EMITTER_GAS.YEAR', 'V_GHG_EMITTER_GAS.ZIP',
       'V_GHG_EMITTER_GAS.FACILITY_NAME', 'V_GHG_EMITTER_GAS.COUNTY_FIPS'],
      dtype='object')

In [143]:
us_greenhouse_gas_emission_direct_emitter_gas_type

,V_GHG_EMITTER_GAS.ADDRESS1,V_GHG_EMITTER_GAS.ADDRESS2,V_GHG_EMITTER_GAS.CITY,V_GHG_EMITTER_GAS.CO2E_EMISSION,V_GHG_EMITTER_GAS.COUNTY,V_GHG_EMITTER_GAS.FACILITY_ID,V_GHG_EMITTER_GAS.GAS_CODE,V_GHG_EMITTER_GAS.GAS_NAME,V_GHG_EMITTER_GAS.LATITUDE,V_GHG_EMITTER_GAS.LONGITUDE,V_GHG_EMITTER_GAS.STATE,V_GHG_EMITTER_GAS.STATE_NAME,V_GHG_EMITTER_GAS.YEAR,V_GHG_EMITTER_GAS.ZIP,V_GHG_EMITTER_GAS.FACILITY_NAME,V_GHG_EMITTER_GAS.COUNTY_FIPS
0,1919 S. BROADWAY,NaN,GREEN BAY,58024.000,BROWN COUNTY,1000589,BIOCO2,Biogenic CO2,44.492500,-88.032300,WI,WISCONSIN,2017,54304,GEORGIA-PACIFIC CONSUMER OPERATIONS LLC,55009.0
1,850 12TH AVE,NaN,NEW YORK,134.500,New York,1000766,CH4,Methane,40.771100,-73.991100,NY,NEW YORK,2018,10019,59th Street,36061.0
2,3379 HWY 482,NaN,Noble,6.854,SABINE,1009343,N2O,Nitrous Oxide,31.621528,-93.724774,LA,LOUISIANA,2012,71462,Baker Road Treater,22085.0
3,1012 BEAUCHAMP ST,NaN,GREENVILLE,53562.000,WASHINGTON COUNTY,1003557,CH4,Methane,33.392476,-91.017584,MS,MISSISSIPPI,2017,38701,TEXAS GAS TRANSMISSION - GREENVILLE STATION,28151.0
4,487 CORN CREEK ROAD,NaN,BEDFORD,7635064.700,Trimble,1006542,CO2,Carbon Dioxide,38.584700,-85.411700,KY,KENTUCKY,2018,40006,Trimble County,21223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219923,14353 McFarland Road,NaN,Burlington,1929.800,SKAGIT COUNTY,1010898,CO2,Carbon Dioxide,48.448041,-122.433615,WA,WASHINGTON,2016,98273,Sierra Pacific Industries - Burlington,53057.0
219924,7628 Thorndike Road,NaN,Greensboro,2776.500,GUILFORD COUNTY,1010033,CO2,Carbon Dioxide,36.075650,-79.965030,NC,NORTH CAROLINA,2018,27409,QORVO,37081.0
219925,NaN,NaN,Calumet,28352.800,CANADIAN COUNTY,1011778,CO2,Carbon Dioxide,35.516370,-98.182280,OK,OKLAHOMA,2014,73014,OFS CS Cedar Lake,40017.0
219926,1186 Calkins Rd,NaN,Troy,23.840,BRADFORD,1010571,N2O,Nitrous Oxide,41.730488,-76.707873,PA,PENNSYLVANIA,2012,16947,Granville Compressor Station,42015.0


In [171]:
US_GH_gas_emissions = us_greenhouse_gas_emission_direct_emitter_gas_type[["V_GHG_EMITTER_GAS.STATE_NAME", "V_GHG_EMITTER_GAS.COUNTY", "V_GHG_EMITTER_GAS.CITY", "V_GHG_EMITTER_GAS.CO2E_EMISSION", "V_GHG_EMITTER_GAS.GAS_CODE", "V_GHG_EMITTER_GAS.GAS_NAME", "V_GHG_EMITTER_GAS.YEAR"]]

In [185]:
pd.options.mode.chained_assignment = None

# Renaming columns and making sure the names match with the final_data set we will eventually merge with.
US_GH_gas_emissions.columns = ["State Name", "County Name", "City Name", "CO2 equiv emission", "Gas Code", "Gas Name", "Year"]
US_GH_gas_emissions["State Name"] = US_GH_gas_emissions["State Name"].str.title()
US_GH_gas_emissions["County Name"] = US_GH_gas_emissions["County Name"].str.title()
US_GH_gas_emissions["City Name"] = US_GH_gas_emissions["City Name"].str.title()

In [186]:
US_GH_gas_emissions

,State Name,County Name,City Name,CO2 equiv emission,Gas Code,Gas Name,Year
0,Wisconsin,Brown County,Green Bay,58024.000,BIOCO2,Biogenic CO2,2017
1,New York,New York,New York,134.500,CH4,Methane,2018
2,Louisiana,Sabine,Noble,6.854,N2O,Nitrous Oxide,2012
3,Mississippi,Washington County,Greenville,53562.000,CH4,Methane,2017
4,Kentucky,Trimble,Bedford,7635064.700,CO2,Carbon Dioxide,2018
...,...,...,...,...,...,...,...
219923,Washington,Skagit County,Burlington,1929.800,CO2,Carbon Dioxide,2016
219924,North Carolina,Guilford County,Greensboro,2776.500,CO2,Carbon Dioxide,2018
219925,Oklahoma,Canadian County,Calumet,28352.800,CO2,Carbon Dioxide,2014
219926,Pennsylvania,Bradford,Troy,23.840,N2O,Nitrous Oxide,2012


In [202]:
Grouped_US_GH_gas_emissions = US_GH_gas_emissions.groupby(["State Name", "County Name", "City Name", "Gas Code"])["CO2 equiv emission"].agg("mean").reset_index()

In [203]:
# CO2 equiv emissions are the average annual CO2 equiv emissions for the gas.
Grouped_US_GH_gas_emissions

,State Name,County Name,City Name,Gas Code,CO2 equiv emission
0,Alabama,Autauga,Autaugaville,CH4,9.502500e+02
1,Alabama,Autauga,Autaugaville,CO2,2.049189e+06
2,Alabama,Autauga,Autaugaville,N2O,1.132698e+03
3,Alabama,Autauga,Billingsley,CH4,6.356875e+02
4,Alabama,Autauga,Billingsley,CO2,1.375766e+06
...,...,...,...,...,...
23017,Wyoming,Weston,Osage,CO2,7.340513e+04
23018,Wyoming,Weston,Osage,N2O,1.081740e+03
23019,Wyoming,Weston County,Newcastle,CH4,1.263103e+03
23020,Wyoming,Weston County,Newcastle,CO2,9.864694e+04


In [204]:
# Convert to daily GH gas emissions
Grouped_US_GH_gas_emissions["Average Daily CO2 equiv emission"] = Grouped_US_GH_gas_emissions["CO2 equiv emission"] / 366

In [207]:
Grouped_US_GH_gas_emissions = Grouped_US_GH_gas_emissions.drop(columns="CO2 equiv emission")

In [208]:
Grouped_US_GH_gas_emissions

,State Name,County Name,City Name,Gas Code,Average Daily CO2 equiv emission
0,Alabama,Autauga,Autaugaville,CH4,2.596311
1,Alabama,Autauga,Autaugaville,CO2,5598.875820
2,Alabama,Autauga,Autaugaville,N2O,3.094803
3,Alabama,Autauga,Billingsley,CH4,1.736851
4,Alabama,Autauga,Billingsley,CO2,3758.924495
...,...,...,...,...,...
23017,Wyoming,Weston,Osage,CO2,200.560474
23018,Wyoming,Weston,Osage,N2O,2.955574
23019,Wyoming,Weston County,Newcastle,CH4,3.451101
23020,Wyoming,Weston County,Newcastle,CO2,269.527162


In [253]:
Pivot_Grouped_US_GH_gas_emissions = Grouped_US_GH_gas_emissions.pivot_table('Average Daily CO2 equiv emission', ["State Name", "County Name", "City Name"], "Gas Code").reset_index()

In [254]:
Pivot_Grouped_US_GH_gas_emissions

Gas Code,State Name,County Name,City Name,BIOCO2,CH4,CO2,HFC,HFE,N2O,NF3,Other,Other_Full,PFC,SF6,Very_Short
0,Alabama,Autauga,Autaugaville,NaN,2.596311,5598.875820,NaN,NaN,3.094803,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,Autauga,Billingsley,NaN,1.736851,3758.924495,NaN,NaN,2.072728,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,Autauga,Prattville,4362.405137,72.520516,860.282705,NaN,NaN,31.186514,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,Baldwin County,Summerdale,NaN,499.213388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,Barbour County,Eufaula,NaN,0.055024,105.047787,NaN,NaN,0.065589,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7672,Wyoming,Uinta County,Mountain View,NaN,5.354679,36.397404,NaN,NaN,0.019948,NaN,NaN,NaN,NaN,NaN,NaN
7673,Wyoming,Washakie,Worland,NaN,0.038934,66.135656,NaN,NaN,0.054552,NaN,NaN,NaN,NaN,NaN,NaN
7674,Wyoming,Washakie County,Worland,NaN,0.047558,79.295663,NaN,NaN,0.067885,NaN,NaN,NaN,NaN,NaN,NaN
7675,Wyoming,Weston,Osage,NaN,0.155055,200.560474,NaN,NaN,2.955574,NaN,NaN,NaN,NaN,NaN,NaN


In [264]:
# Resetting index
Pivot_Grouped_US_GH_gas_emissions = Pivot_Grouped_US_GH_gas_emissions.rename_axis(None, axis=1)

In [266]:
Pivot_Grouped_US_GH_gas_emissions

,State Name,County Name,City Name,BIOCO2,CH4,CO2,HFC,HFE,N2O,NF3,Other,Other_Full,PFC,SF6,Very_Short
0,Alabama,Autauga,Autaugaville,NaN,2.596311,5598.875820,NaN,NaN,3.094803,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,Autauga,Billingsley,NaN,1.736851,3758.924495,NaN,NaN,2.072728,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,Autauga,Prattville,4362.405137,72.520516,860.282705,NaN,NaN,31.186514,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,Baldwin County,Summerdale,NaN,499.213388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,Barbour County,Eufaula,NaN,0.055024,105.047787,NaN,NaN,0.065589,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7672,Wyoming,Uinta County,Mountain View,NaN,5.354679,36.397404,NaN,NaN,0.019948,NaN,NaN,NaN,NaN,NaN,NaN
7673,Wyoming,Washakie,Worland,NaN,0.038934,66.135656,NaN,NaN,0.054552,NaN,NaN,NaN,NaN,NaN,NaN
7674,Wyoming,Washakie County,Worland,NaN,0.047558,79.295663,NaN,NaN,0.067885,NaN,NaN,NaN,NaN,NaN,NaN
7675,Wyoming,Weston,Osage,NaN,0.155055,200.560474,NaN,NaN,2.955574,NaN,NaN,NaN,NaN,NaN,NaN


In [268]:
# Get number of non null entires (to see which gasses to add to the data set)
Pivot_Grouped_US_GH_gas_emissions.count()

State Name     7677
County Name    7677
City Name      7677
BIOCO2         1360
CH4            7599
CO2            6908
HFC              91
HFE              36
N2O            6507
NF3              63
Other            17
Other_Full       69
PFC              96
SF6             226
Very_Short       50
dtype: int64

## Based on the counts of non null entries, we have a reasonable amount of data for BIOCO2, CH4, CO2, and N2O. So we will try to use these as features. We can revist this if need be.

In [272]:
# Truncating data to the gasses we want and filling the NaN with 0s
Truncated_US_GH_gas_emissions = Pivot_Grouped_US_GH_gas_emissions[["State Name", "County Name", "City Name", "BIOCO2", "CH4", "CO2", "N2O"]].fillna(0)

In [273]:
Truncated_US_GH_gas_emissions

,State Name,County Name,City Name,BIOCO2,CH4,CO2,N2O
0,Alabama,Autauga,Autaugaville,0.000000,2.596311,5598.875820,3.094803
1,Alabama,Autauga,Billingsley,0.000000,1.736851,3758.924495,2.072728
2,Alabama,Autauga,Prattville,4362.405137,72.520516,860.282705,31.186514
3,Alabama,Baldwin County,Summerdale,0.000000,499.213388,0.000000,0.000000
4,Alabama,Barbour County,Eufaula,0.000000,0.055024,105.047787,0.065589
...,...,...,...,...,...,...,...
7672,Wyoming,Uinta County,Mountain View,0.000000,5.354679,36.397404,0.019948
7673,Wyoming,Washakie,Worland,0.000000,0.038934,66.135656,0.054552
7674,Wyoming,Washakie County,Worland,0.000000,0.047558,79.295663,0.067885
7675,Wyoming,Weston,Osage,0.000000,0.155055,200.560474,2.955574


# Merging Gas data with our Final Features Dataset

In [283]:
## Merging with the final data set
complete_data_set = final_data_set.merge(Truncated_US_GH_gas_emissions, on=["State Name", "County Name", "City Name"])

In [284]:
complete_data_set

,State Code,County Code,Site Num,State Name,County Name,City Name,Latitude,Longitude,Date Local,Mean Temperature (F),Mean Wind Direction (compass degrees),Mean Wind Speed (knots),Elevation,Land Use,Location Setting,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,BIOCO2,CH4,CO2,N2O
0,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.8150,2020-01-09,52.125000,137.125000,4.904167,177.0,COMMERCIAL,URBAN AND CENTER CITY,56,Moderate,PM2.5,01-073-0023,9,0.0,194.591845,145.217054,0.555493
1,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.8150,2020-01-10,65.750000,163.708333,7.583333,177.0,COMMERCIAL,URBAN AND CENTER CITY,44,Good,PM2.5,01-073-0023,9,0.0,194.591845,145.217054,0.555493
2,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.8150,2020-01-14,63.937500,208.916667,3.166667,177.0,COMMERCIAL,URBAN AND CENTER CITY,30,Good,NO2,01-073-0023,9,0.0,194.591845,145.217054,0.555493
3,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.8150,2020-01-16,58.012500,75.458333,5.945833,177.0,COMMERCIAL,URBAN AND CENTER CITY,31,Good,Ozone,01-073-0023,9,0.0,194.591845,145.217054,0.555493
4,1,73,23,Alabama,Jefferson,Birmingham,33.553056,-86.8150,2020-01-17,54.945833,105.375000,5.491667,177.0,COMMERCIAL,URBAN AND CENTER CITY,34,Good,Ozone,01-073-0023,8,0.0,194.591845,145.217054,0.555493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30140,56,13,4,Wyoming,Fremont,Riverton,43.024210,-108.3637,2020-09-18,59.291667,195.166667,2.687500,1500.0,RESIDENTIAL,SUBURBAN,106,Unhealthy for Sensitive Groups,PM2.5,56-013-0004,5,0.0,9.998634,167.363388,0.091191
30141,56,13,4,Wyoming,Fremont,Riverton,43.024210,-108.3637,2020-09-19,61.708333,215.166667,4.508333,1500.0,RESIDENTIAL,SUBURBAN,60,Moderate,PM2.5,56-013-0004,4,0.0,9.998634,167.363388,0.091191
30142,56,13,4,Wyoming,Fremont,Riverton,43.024210,-108.3637,2020-10-05,56.416667,224.500000,3.720833,1500.0,RESIDENTIAL,SUBURBAN,54,Moderate,PM2.5,56-013-0004,4,0.0,9.998634,167.363388,0.091191
30143,56,13,4,Wyoming,Fremont,Riverton,43.024210,-108.3637,2020-10-11,50.333333,258.958333,10.012500,1500.0,RESIDENTIAL,SUBURBAN,52,Moderate,PM10,56-013-0004,4,0.0,9.998634,167.363388,0.091191


In [285]:
# Non-zero count for each column
np.count_nonzero(complete_data_set, axis=0)

array([30145, 30145, 30145, 30145, 30145, 30145, 30145, 30145, 30145,
       30145, 30145, 30145, 28016, 30145, 30145, 28558, 30145, 30145,
       30145, 30145,  7076, 29787, 28599, 28421])

## For this final aggregated dataset the BIOCO2, CH4, CO2, and N2O average daily emissions non-zero counts are 7076, 29787, 28599, and 28421, respectively. These values seem reasonable but if the lower values for BIOCO2 cause issues then we can remove that feature. 

## From this EDA our final dataset is complete_data_set, and has the following features: State Name, County Name, City Name, Mean Temperature (F), Mean Wind Direction (compass degrees), Mean Wind Speed (knots), Elevation, Land Use, Location Setting, BIOCO2, CH4, CO2, and N2O. The dataset also contains the AQI which is what we will be trying to predict, along with some other columns which can be considered meta data.

## If time permits precipitation from the daily_global_weather_2020 should be added.